# Cálculo numérico de la serie de Fourier

Federico Cerisola (cerisola@df.uba.ar)

Departamento de Física, Facultad de Ciencias Exactas y Naturales, Universidad de Buenos Aires

In [9]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate

from numpy import sin, cos, tan, exp, log, cosh, sinh, sqrt, pi, piecewise

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

import ipywidgets as widgets
import parser

En este notebook se muestra como calcular numéricamente los términos de la serie de Fourier de una función cualquiera.

En la notación del código, expresamos el desarrollo fourier de una función $f$ periódica con período $T$ como

$$
f(t) = \frac{B_0}{2} + \sum_{n=1}^{\infty} A_n\sin\left(\frac{2\pi n t}{T}\right) + B_n\cos\left(\frac{2\pi n t}{T}\right)
$$

In [10]:
# Esta función toma como input:
# . la función a desarrollar:'f'
# . el período de la función: 'T'
# . el número de coeficiente que acompaña a los senos: n
# y devuelve el coeficiente An
def coef_fourier_sin(f, n, T=1):
    integrando = lambda t: f(t)*np.sin(2*np.pi*n*t/T)
    An = (2 / T) * integrate.quad(integrando, 0, T)[0]
    return An


# Esta función toma como input:
# . la función a desarrollar:'f'
# . el período de la función: 'T'
# . el número de coeficiente que acompaña a los cosenos: n
# y devuelve el coeficiente Bn
def coef_fourier_cos(f, n, T=1):
    integrando = lambda t: f(t)*np.cos(2*np.pi*n*t/T)
    Bn = (2 / T) * integrate.quad(integrando, 0, T)[0]
    return Bn

# Esta función toma como input:
# . la función a desarrollar:'f'
# . el período de la función: 'T'
# . el número maximo hasta el cual calcular los coeficientes del desarrollo: 'nmax'
# y devuelve dos vectores con los coeficientes: An, Bn (que corresponden a los senos y cosenos respectivamente)
def calcular_coefs_fourier(f, nmax, T=1):
    A = np.zeros(nmax+1)
    B = np.zeros(nmax+1)
    for n in range(nmax+1):
        A[n] = coef_fourier_sin(f, n, T)
        B[n] = coef_fourier_cos(f, n, T)
    A[0] = 0
    B[0] = B[0] / 2
    return A, B


# Esta función toma como input:
# . la función a desarrollar:'f'
# . el período de la función: 'T'
# . el número maximo hasta el cual calcular los coeficientes del desarrollo: 'nmax'
# y devuelve una función 'fapprox' que es la aproximación a 'f'
# a la función 'fapprox' se la puede usar como a cualquier función, simplemente evaluando 'f(t)'
def calcular_approx_fourier(f, nmax, T=1, return_coefs=False):
    A, B = calcular_coefs_fourier(f, nmax, T)
    def fapprox(t):
        r = 0
        for n, (An, Bn) in enumerate(zip(A, B)):
            r += An*sin(2*np.pi*n*t/T) + Bn*cos(2*np.pi*n*t/T)
        return r
    
    if return_coefs:
        return fapprox, A, B
    else:
        return fapprox

In [11]:
# Esta función toma como input:
# . la función a desarrollar:'f'
# . el período de la función: 'T'
# . el número maximo hasta el cual calcular los coeficientes del desarrollo: 'nmax'
# y utilizando las funciones anteriores grafica la función exacta y la aproximación de Fourier calculada
def graficar_approx_fourier(f, nmax, T=1, nperiodos=1):
    ffourier, A, B = calcular_approx_fourier(f, nmax, T, return_coefs=True)

    t = np.linspace(0, T, 400)
    fexacta = f(t)
    text = np.linspace(0, nperiodos*T, nperiodos*t.size)
    fexactaext = np.concatenate([fexacta]*nperiodos)
    fapprox = ffourier(text)

    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,10))

    ax[0].plot(text, fexactaext, '-', color='C0', linewidth=3.0, label='exacta')
    ax[0].plot(text, fapprox, '-', color='C1', linewidth=3.0,
               label='approx. Fourier ($n_{{max}}$ = {})'.format(nmax))
    ax[0].grid()
    ax[0].set_xlabel('$t$')
    ax[0].legend()

    for n in range(A.size):
        if n == 0: continue
        ax[1].plot(text, A[n]*sin(2*np.pi*n*text/T), '-', color='C{}'.format(n-1), linewidth=3.0)
        ax[2].plot(text, B[n]*cos(2*np.pi*n*text/T), '-', color='C{}'.format(n-1), linewidth=3.0)

    fig.tight_layout()
    plt.show()
    
    print('A = ', A)
    print('B = ', B)


# Esta función toma como input:
# . la función a desarrollar en texto:'ftext'
# . el período de la función: 'T'
# . el número maximo hasta el cual calcular los coeficientes del desarrollo: 'nmax'
# y parsea el texto en una función de python y luego grafica los resultados usando la función anterior
def graficar_approx_fourier_customf(ftext, nmax, T=1, nperiodos=1):
    fcode = parser.expr(ftext).compile()
    f = lambda t: eval(fcode)
    graficar_approx_fourier(f, nmax, T, nperiodos)

In [13]:
widgets.interact(graficar_approx_fourier_customf,
                 ftext=widgets.Textarea(value='t', description='Función'),
                 nmax=widgets.IntSlider(min=1, max=80, step=1, value=1, description='n máximo'),
                 T=widgets.fixed(1), nperiodos=widgets.fixed(1))

interactive(children=(Textarea(value='t', description='Función'), IntSlider(value=1, description='n máximo', m…

<function __main__.graficar_approx_fourier_customf(ftext, nmax, T=1, nperiodos=1)>

En el widget interactivo anterior pueden elegir hasta qué orden calcular el desarrollo y además pueden escribir la función que quieren para desarrollar.

Para escribir funciones, la sintáxis es relativamente sencilla, con '\*' hacen el producto, con '\*\*' la potencia (i.e. 't\*\*2' es $t^2$), con '/' la división y reconoce funciones básica tipo: 'sin', 'cos', 'tan', 'exp', 'log', 'cosh', 'sinh', 'sqrt'. Algunos ejemplos que pueden a probar a poner en el cuadro de texto de la función:

* t
* t\*\*2
* 0.25-(t-0.5)\*\*2
* (2\*t + t\*\*2)\*cos(2\*pi\*t)
* 2-cosh(t-0.5)

etc.

Pueden también definir fnciones a trozos, aunque en ese caso la sintáxis es un poco más complicada. Acá algunos ejemplos, pueden probar a jugar modificándolos

* piecewise(t, [t < 0.5, t >= 0.5], [0, 1])
* piecewise(t, [t < 0.5, t >= 0.5], [0, lambda t: t-0.5])
* piecewise(t, [t < 0.5, t >= 0.5], [lambda t: t, lambda t: 1-t])
* piecewise(t, [t <= 0.25, (t > 0.25) & (t < 0.5), (t >= 0.5) & (t < 0.75) ,t >= 0.75], [0, lambda t: t - 0.25, lambda t: 0.75-t, 0])

Algunos ejemplos de las guías:

* piecewise(t, [t < 0.25, t >= 0.75], [0, 0, 1])
* piecewise(t, [t < 0.15, t >= 0.35], [0, 0, 1])
* piecewise(t, [t <= 0.25, (t > 0.25) & (t < 0.5), (t >= 0.5) & (t < 0.75) ,t >= 0.75], [0, lambda t: t - 0.25, lambda t: 0.75-t, 0])
* piecewise(t, [t <= 0.25, (t >= 0.25) & (t <= 0.75), t >= 0.75], [lambda t: -t, lambda t: -0.5 + t, lambda t: -t + 1.0])
* piecewise(t, [t <= 0.75], [lambda t: t, lambda t: -(t - 1)*0.75/0.25])